In [35]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer

from pyspark.sql.functions import col, explode
import pandas as pd
import pyspark.sql.functions as func
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark import SparkContext
# import findspark 
# findspark.init()  
# import pyspark

In [36]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "16G").getOrCreate()
#for later --conf spark.sql.catalogImplementation=hive 

In [37]:
trainSample = spark.read.parquet('train_sample1.parquet')
testSample = spark.read.parquet('test_sample1.parquet')
trainSample.createOrReplaceTempView('trainSample')
testSample.createOrReplaceTempView('testSample')

In [6]:
valSample = spark.read.parquet('val_sample1.parquet')

In [5]:
indexer_obj_1 = StringIndexer(inputCol="user_id", outputCol="user_id_numer").setHandleInvalid("keep")
indexer_model_1 = indexer_obj_1.fit(trainSample)
indexer_df_1 = indexer_model_1.transform(trainSample)

indexer_obj_2 = StringIndexer(inputCol="track_id", outputCol="track_id_numer").setHandleInvalid("keep")
indexer_model_2= indexer_obj_2.fit(indexer_df_1)
indexer_df_2 = indexer_model_2.transform(indexer_df_1)

train_df = indexer_df_2.drop('user_id')
train_df= train_df.drop('track_id')


In [7]:
val_df_1 = indexer_model_1.transform(valSample)
val_df_2= indexer_model_2.transform(val_df_1)

val_df = val_df_2.drop('user_id')
val_df= val_df.drop('track_id')

In [8]:
val_df

DataFrame[count: bigint, __index_level_0__: bigint, user_id_numer: double, track_id_numer: double]

In [9]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="user_id_numer", itemCol="track_id_numer", ratingCol="count",
          coldStartStrategy="drop")
model = als.fit(train_df)

#userRecs = model.recommendForAllUsers(10)

# Evaluate the model by computing the RMSE on the test data

# evaluator = RegressionEvaluator(metricName="rmse", labelCol="count",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)

# print("Root-mean-square error = " + str(rmse))

# Generate top 10 song recommendations for each user

### ADJUST FOR 500 SONGS 

# Generate top 10 user recommendations for each song

#songRecs = model.recommendForAllItems(10)

In [ ]:
#### THIS WILL BE REMOVED ####

# Generate top 10 movie recommendations for a specified set of users
users = train_df.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# Generate top 10 user recommendations for a specified set of movies
# songs = df_dropped.select(als.getItemCol()).distinct().limit(3)
# songSubSetRecs = model.recommendForItemSubset(songs, 10)

In [ ]:
## MIGHT NOT NEED ##

predictions = model.transform(test_df)
predictions.orderBy('prediction', ascending=False)

In [10]:
# for each user, sort track ids by count
val_true = val_df.orderBy('count')

# flatten to group by user id and get list of true track ids
val_true_flatten = val_true.groupby('user_id_numer').agg(func.collect_list('track_id_numer').alias("track_id_numer"))

# add to dictionary
val_true_dict = val_true_flatten.collect()
val_true_dict = [{r['user_id_numer']: r['track_id_numer']} for r in val_true_dict]
val_true_dict = dict((key,d[key]) for d in val_true_dict for key in d)
val_true_dict

{144345.0: [2576.0],
 257834.0: [5251.0, 1748.0],
 303341.0: [106998.0],
 279263.0: [5665.0],
 259239.0: [6919.0],
 231190.0: [106998.0],
 293951.0: [2662.0, 85544.0],
 159194.0: [4265.0],
 220558.0: [106998.0],
 321044.0: [18028.0],
 289159.0: [508.0],
 120784.0: [64175.0],
 314898.0: [21924.0, 28231.0],
 216365.0: [23.0, 69.0, 18.0],
 123501.0: [53918.0],
 223776.0: [71.0],
 147473.0: [7539.0, 25750.0],
 75511.0: [37293.0],
 304923.0: [22201.0],
 261625.0: [7574.0],
 295799.0: [40.0],
 157069.0: [106998.0],
 230320.0: [16226.0],
 292319.0: [9474.0],
 129140.0: [106998.0],
 111524.0: [9113.0],
 202200.0: [34735.0],
 265663.0: [975.0],
 290401.0: [29.0, 1.0, 9636.0],
 103678.0: [85936.0],
 213052.0: [12727.0],
 112524.0: [4.0],
 324257.0: [171.0],
 326013.0: [14874.0],
 309446.0: [63374.0],
 133623.0: [1013.0],
 270912.0: [22723.0],
 252645.0: [9436.0],
 76493.0: [5219.0],
 105880.0: [7848.0],
 305804.0: [11.0],
 193744.0: [5300.0],
 168872.0: [4099.0],
 320425.0: [5439.0],
 198823.0: 

In [ ]:
#https://stackoverflow.com/questions/59390481/how-to-implement-ranking-metrics-of-pyspark

In [11]:
val_preds = model.recommendForUserSubset(val_df,10)
val_preds_explode = val_preds.select(val_preds.user_id_numer,explode(val_preds.recommendations.track_id_numer))

val_preds_flatten = val_preds_explode.groupby('user_id_numer').agg(func.collect_list('col').alias("col"))

val_preds_dict = val_preds_flatten.collect()
val_preds_dict = [{r['user_id_numer']: r['col']} for r in val_preds_dict]
val_preds_dict = dict((key,d[key]) for d in val_preds_dict for key in d)

Evaluation Metric

In [41]:
labels_list = []

for user in val_preds_dict.keys():
    labels_list.append((val_preds_dict[user], [int(i) for i in val_true_dict[user]]))

labels = spark.sparkContext.parallelize(labels_list)
#metrics = pyspark.mllib.evaluation.RankingMetrics(labels)
#print(metrics.meanAveragePrecision)
    
# add predicted items and true items as tuples to list

# parallelize list
# call ranking metrics on list 

In [44]:
labels.collect()

[([1757, 50608, 9831, 54627, 48884, 31340, 12756, 4576, 67294, 52190], [5219]),
 ([50608, 16901, 48884, 40636, 4576, 73926, 48512, 22906, 80469, 26092], [63]),
 ([54627, 16999, 11827, 10896, 60450, 12756, 22981, 25176, 80706, 11357],
  [18967]),
 ([11827, 35076, 100229, 50608, 23515, 15646, 10472, 34774, 46876, 15387],
  [56086]),
 ([1757, 16999, 48884, 22981, 80469, 16901, 32946, 11357, 40636, 18604],
  [9113]),
 ([50608, 11827, 1757, 15646, 48512, 34774, 31340, 41276, 36349, 23388],
  [19998]),
 ([4576, 50608, 25296, 92294, 13058, 18604, 72755, 16901, 10702, 9255], [370]),
 ([48884, 73926, 9255, 8603, 10472, 37291, 12756, 52190, 25461, 40636],
  [7370]),
 ([54627, 73926, 28588, 62081, 10896, 60450, 52190, 4576, 26092, 41339],
  [9436]),
 ([54627, 100229, 28588, 62245, 5314, 95422, 46876, 58817, 26092, 67491],
  [18799]),
 ([28588, 73926, 40636, 13058, 72755, 46195, 9255, 16901, 25296, 41339],
  [412]),
 ([11827, 25176, 28588, 100229, 31449, 34774, 62245, 10472, 60450, 49471],
  [7510